In [1]:
import os

In [2]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\Youtube Project\\research'

In [3]:
os.chdir('..')

In [4]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\Youtube Project'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    training_data: Path
    updated_base_model_path: Path
    param_epochs: int
    param_batch_size: int
    param_image_size: list
    param_is_augmentation: bool

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])

    def get_training_config(self) -> TrainingConfig:
        training_config = self.config.training

        root_dir = Path(training_config.root_dir)
        create_directories([root_dir])

        training_data = Path(self.config.data_ingestion.unzip_dir)
        updated_base_model_path = Path(self.config.prepare_base_model.updated_base_model_path)

        param_epochs = self.params.EPOCHS
        param_batch_size = self.params.BATCH_SIZE
        param_image_size = self.params.IMAGE_SIZE
        param_is_augmentation = self.params.AUGMENTATION

        training_config = TrainingConfig(
            root_dir=root_dir,
            trained_model_path=Path(training_config.trained_model_path),
            training_data=training_data,
            updated_base_model_path=updated_base_model_path,
            param_epochs=param_epochs,
            param_batch_size=param_batch_size,
            param_image_size=param_image_size,
            param_is_augmentation=param_is_augmentation
        )

        return training_config

In [7]:
import os
from urllib.request import Request
from zipfile import ZipFile
import tensorflow as tf
import time

In [8]:
class ModelTrainer:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def train_valid_generator(self):
        datagen_kwargs = dict(rescale=1./255, validation_split=0.20)
        dataflow_kwargs = dict(target_size=self.config.param_image_size[:2],
                               batch_size=self.config.param_batch_size,
                               interpolation="bilinear")

        if self.config.param_is_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=20,
                horizontal_flip=True,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.1,
                zoom_range=0.2,
                **datagen_kwargs
            )
        else:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)

        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)

        self.train_generator = train_datagen.flow_from_directory(
            directory=self.config.training_data.as_posix(),
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        self.valid_generator = valid_datagen.flow_from_directory(
            directory=self.config.training_data.as_posix(),
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    def load_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    @staticmethod
    def save_model(model, path: Path):
        model.save(path)
    
    def train(self):
        self.step_per_epoch = self.train_generator.samples // self.config.param_batch_size
        self.validation_steps = self.valid_generator.samples // self.config.param_batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.param_epochs,
            steps_per_epoch=self.step_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
        )
        self.save_model(model=self.model, path=self.config.trained_model_path)

In [ ]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    model_trainer = ModelTrainer(config=training_config)
    model_trainer.load_base_model()
    model_trainer.train_valid_generator()
    model_trainer.train()
except Exception as e:
    raise e